# DATA 620, Project 2

## Goals:
1 - Identify a large 2-node network dataset—you can start with a dataset in a repository. 


2 - Your data should meet the criteria that it consists of ties between and not within two (or more) distinct groups. Reduce the size of the network using a method such as the island method described in chapter 4 of social network analysis.


3 - What can you infer about each of the distinct groups?


Overview
This project will use data from https://snap.stanford.edu/data/web-Movies.html

"This dataset consists of movie reviews from amazon. The data span a period of more than 10 years, including all ~8 million reviews up to October 2012. Reviews include product and user information, ratings, and a plaintext review. We also have reviews from all other Amazon categories."

Nodes: Product (movies) and Reviewers

About the data set:
Number of reviews: 7,911,684   
Number of users: 889,176   
Number of products: 253,059   
Users with > 50 reviews: 16,341   
Median no. of words per review 101   
Timespan Aug 1997 - Oct 2012   


In [2]:
import gzip
import os
import csv
import networkx as net
import matplotlib.pyplot as plot
import math

In [3]:
matplotlib inline

In [4]:
'''
#This section is the code that I used to read in the data and create the graphs and lists.

#The data set is very large so only read in the first 2 million records.
filename = 'movies.txt.gz'

#set up the graph
g=net.Graph()

#set up blank lists
node1 = []
node2 = []
movies = []
reviewers = []

#read data from the file to create the graph and the movies and reviewers list
with gzip.open(filename,'rU') as f:
    reader = csv.reader(f)
    #for line in reader:
    for i in range(2000000):
        line = reader.next()
        if line != []:
            if line[0].split(":",1)[0] == 'product/productId':
                node1 = line[0].split(": ",1)[1]
                if node1 not in movies:
                    movies.append(node1)
            if line[0].split(":",1)[0] == 'review/userId':
                node2 = line[0].split(": ",1)[1]
                if node2 not in reviewers:
                    reviewers.append(node2)
        if node2 != []:
            #create an edge
            g.add_edge(node1,node2)
            #print [node1,node2]
            #reset node1 and node2, if node2 is not blank
            node1 = []
            node2 = []
'''

'\n#This section is the code that I used to read in the data and create the graphs and lists.\n\n#The data set is very large so only read in the first 2 million records.\nfilename = \'movies.txt.gz\'\n\n#set up the graph\ng=net.Graph()\n\n#set up blank lists\nnode1 = []\nnode2 = []\nmovies = []\nreviewers = []\n\n#read data from the file to create the graph and the movies and reviewers list\nwith gzip.open(filename,\'rU\') as f:\n    reader = csv.reader(f)\n    #for line in reader:\n    for i in range(2000000):\n        line = reader.next()\n        if line != []:\n            if line[0].split(":",1)[0] == \'product/productId\':\n                node1 = line[0].split(": ",1)[1]\n                if node1 not in movies:\n                    movies.append(node1)\n            if line[0].split(":",1)[0] == \'review/userId\':\n                node2 = line[0].split(": ",1)[1]\n                if node2 not in reviewers:\n                    reviewers.append(node2)\n        if node2 != []:\n   

In [5]:
## Import bi-partite (bi-modal) functions
from networkx.algorithms import bipartite as bi

In [6]:
#write the graph to a file so it doesn't have to be re-created every time
#net.write_pajek(g, "movie_reviews.net")
#have to write the list of movies and reviewers to a file as well
g=net.read_pajek("movie_reviews.net")

In [7]:
#pajek brings in multi-graph, need to convert it
g = net.Graph(g)

In [37]:
'''
#write movie and reviewer lists to csv files
with open('movies.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(movies)

with open('reviewers.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(reviewers)

'''
#read in movie and reviewer lists
with open('movies.csv', 'rU') as f:
    reader = csv.reader(f)
    movies = list(reader)
    movies = movies[0]
    
with open('reviewers.csv', 'rU') as f:
    reader = csv.reader(f)
    reviewers = list(reader)
    reviewers = reviewers[0]

In [38]:
[len(movies),len(reviewers)]

[6816, 124500]

In [11]:
#project the graph onto movies
M = bi.projected_graph(g, movies)

In [12]:
#check the number of subgraphs
len(list(net.connected_component_subgraphs(M)))

1603

In [14]:
[len(c) for c in net.connected_component_subgraphs(M)
if len(c) > 5]

[5144, 6]

In [19]:
#Compute an affiliation network of the movies and reviews:
movienet=bi.weighted_projected_graph(g, movies, ratio=False)

In [43]:
len(movienet.nodes())

6816

In [20]:
len(list(net.connected_component_subgraphs(movienet)))

1603

In [56]:
#number of component subgraphs of size 1
len([len(c) for c in net.connected_component_subgraphs(movienet)
if len(c) < 2])

1544

In [57]:
#number of component subgraphs of size 2
len([len(c) for c in net.connected_component_subgraphs(movienet)
if len(c) == 2])

50

In [58]:
#Number of component subgraphs of size greater than 2
[len(c) for c in net.connected_component_subgraphs(movienet)
if len(c) > 2]

[5144, 3, 4, 3, 3, 6, 3, 3, 3]

The movie and reviewer network has over 6800 nodes, but the network is split into 1603 component subgraphs.  Of these components, 1544 are of size 1. These are considered isolates and should be removed from the network.  There are 50 components of size 2. Leaving only 9 component subgraphs of greater than size 2, of which one is of size 5144 and the remaining of size 6 or less. We will treat the large component network of size 5144 as the whole network.  However, this subgraph is still very large.

Implement the Island Method  
  
The first thing we need to implement for the island method is a function to virtually raise the water level. The function below takes a graph, and applies a threshold (“water level”), letting all edges above a certain value through, and removing all others.  
  
Then define how the water level should be raised. Compute evenly spaced thresholds and produce a list of networks at each water level:

In [15]:
def trim_edges(g, weight=1):
    g2=net.Graph()
    for f, to, edata in g.edges(data=True):
        if edata['weight'] > weight:
            g2.add_edge(f,to,edata)
    return g2

In [16]:
#This function will return a list of graph objects, each corresponding to a specific water level.
def island_method(g, iterations=5):
    weights= [edata['weight'] for f,to,edata in g.edges(data=True)]
    mn=int(min(weights))
    mx=int(max(weights))
    #compute the size of the step, so we get a reasonable step in iterations
    step=int((mx-mn)/iterations)
    return [[threshold, trim_edges(g, threshold)] for threshold in range(mn,mx,step)]

Isolate the biggest component of movie review network and separate it into subparts using the island method:

In [22]:
cc=list(net.connected_component_subgraphs(movienet))[0]

In [23]:
islands=island_method(cc)

In [24]:
for i in islands:
    print i[0],len(i[1]),len(list(net.connected_component_subgraphs(i[1])))
# print the threshold level, size of the graph, and number of connected components

1 3007 89
151 106 49
301 43 20
451 17 8
601 6 3
751 2 1


This is interpreted as: when all links with a value of 1 are dropped, the network separates into 89 island subgraphs, each representing a set of movies with common reviewers.  
  
Threshholding at 151, there are 106 nodes left in 49 islands.
Thresholding at 301, there are 43 nodes left in 20 islands.

In [63]:
islands[2]

[301, <networkx.classes.graph.Graph at 0x7c477f0>]

In [66]:
islands301 = islands[2][1]

In [73]:
#import code that was downloaded from the code for chapter 4 in the book
import triadic
import draw_triads

In [71]:
## Run the triadic census
census, node_census = triadic.triadic_census(islands301)

In [72]:
census

{'003': 11281,
 '012': 0,
 '021C': 0,
 '021D': 0,
 '021U': 0,
 '030C': 0,
 '030T': 0,
 '102': 1057,
 '111D': 0,
 '111U': 0,
 '120C': 0,
 '120D': 0,
 '120U': 0,
 '201': 0,
 '210': 0,
 '300': 3}

In [74]:
## get only the number of closed triads, and sort the list by the value, descending
closed_triads=[[-k,v] for k,v in sorted([[-node_census[k]['300'],k] for k in
node_census.keys()])]


In [76]:
closed_triads

[[1, u'0790701251'],
 [1, u'B000J0XJC2'],
 [1, u'B000O76T7M'],
 [0, u'0792101421'],
 [0, u'7883704540'],
 [0, u'B00000F168'],
 [0, u'B00004CLDC'],
 [0, u'B00004VYLU'],
 [0, u'B000055XP5'],
 [0, 'B00005JMZK'],
 [0, u'B00005LKLD'],
 [0, u'B00005LL26'],
 [0, u'B00005MFO8'],
 [0, u'B000065MQO'],
 [0, u'B000067JG3'],
 [0, u'B000067JG4'],
 [0, u'B00006LLJ4'],
 [0, u'B000071ZZI'],
 [0, u'B0000DK4QJ'],
 [0, u'B00015HX90'],
 [0, u'B0006IWQIU'],
 [0, u'B0007A2GSM'],
 [0, u'B0007A2GSW'],
 [0, 'B0007OCG4W'],
 [0, u'B0007Y08II'],
 [0, u'B0007Y08IS'],
 [0, u'B000B8VCSA'],
 [0, u'B000HOMU98'],
 [0, 'B000IMM3XW'],
 [0, u'B000NQRV4O'],
 [0, u'B000VBJEFK'],
 [0, u'B000ZLFALI'],
 [0, u'B000ZLFALS'],
 [0, u'B001FB55HQ'],
 [0, u'B001G7Q0Z0'],
 [0, u'B00288KNLS'],
 [0, u'B0028OA3EO'],
 [0, u'B0028OA3EY'],
 [0, u'B002OHDRF2'],
 [0, u'B002SEQ8ZM'],
 [0, u'B002ZHKZCY'],
 [0, u'B0032LO8DE'],
 [0, u'B0068FZ05Q']]

No movie is part of more than one triad (cliques), and only three movies have even one triad

In [77]:
cliques = list(net.find_cliques(islands301))

In [79]:
len(cliques)

20

In [80]:
cliques

[[u'0790701251', u'B000065MQO', u'B000HOMU98'],
 [u'B0032LO8DE', u'B00000F168'],
 [u'B000O76T7M', u'B0006IWQIU', 'B00005JMZK'],
 [u'B000055XP5', 'B000IMM3XW'],
 [u'B00006LLJ4', u'B0000DK4QJ'],
 [u'B00005LL26', u'B00005LKLD'],
 [u'B000J0XJC2', u'B00015HX90', u'0792101421'],
 [u'B000NQRV4O', 'B0007OCG4W'],
 [u'B0007Y08IS', u'B0007Y08II'],
 [u'B001G7Q0Z0', u'B00005MFO8'],
 [u'B00288KNLS', u'B002SEQ8ZM'],
 [u'B0028OA3EY', u'B0028OA3EO'],
 [u'B000067JG4', u'B000067JG3'],
 [u'B000VBJEFK', u'7883704540'],
 [u'B000ZLFALI', u'B000ZLFALS'],
 [u'B002OHDRF2', u'B001FB55HQ'],
 [u'B0068FZ05Q', u'B000071ZZI'],
 [u'B0007A2GSW', u'B0007A2GSM'],
 [u'B002ZHKZCY', u'B000B8VCSA'],
 [u'B00004VYLU', u'B00004CLDC']]

Cliques shows us the movies in the cliques, and the three triads are obvious here.

The implication is there isn't a lot of inteconnetedness between movies with resepect to common reviewers.  There are occasional movies that get reviewed by the same sets of reviewers and these same reviewers review other movies as well, but this isn't very common.